In [259]:
import networkx as nx
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt
import gurobipy as grb
import seaborn as sns

# Scheduling algorithm

Some global parameters

In [260]:
n_trays = 8
n_holes_per_tray = 5
horizon = 100
max_size = 15

Represent a node, either a hole, sink or source

In [261]:
class Node(object):
    def __init__(self,*args) :
        
        #('hole', tray, hole, when)
        if args[0] == 'hole' :
            self.where = args[1]* (n_holes_per_tray + 2) + args[2]
            self.when = args[3]
            self.sink = False
            self.type = 'hole'
            
        #('source', where, when)
        elif args[0] == 'source':
            self.where = args[1]
            self.when = args[2]
            self.sink = False
            self.type = 'source'
        
        #('sink', where, when)
        elif args[0] == 'sink' :
            self.where = args[1]
            self.when = args[2]
            self.sink = True
            self.type = 'sink'
    
    def neighbors(self,other):
        return self.type == 'hole' and other.type == 'hole' and self.when == other.when and abs(self.where - other.where) == 1
    
    def __eq__(self,other):
        return self.where == other.where and self.when == other.when and self.type == other.type
    
    def __ne__(self,other):
        return not eq(self,other)
    
    def __hash__(self):
        return hash((self.where, self.when,self.type))

In [262]:
'''
bounds : (plants, value)  value = [0,1]
size : int, size of the active plant
'''

Edge = namedtuple('Edge',['node_from', 'node_to','bounds','size'])

In [263]:
'''
#classic plant data
plant_data = {
    'Lettuce': {
        'total_days': 8,
        'transfers': [(0, 1)],
        0: (0, 2),
        1: (2, 8),
        'color': 'b',
        'size' : (1,2,3,4,5,6,7,8),
    },
    'Fennel': {
        'total_days': 5,
        'transfers': [(1, 0)],
        1: (0, 3),
        0: (3, 5),
        'color': 'g',
        'size' : (1,1,3,4,6),
    },
    'Marijuana': {
        'total_days': 5,
        'transfers': [(2, 3 ,4 , 5)],
        2: (0, 1),
        3: (1, 2),
        4: (2,3),
        5: (3,5),
        'color': 'r',
        'size' : (1,2,3,4,4),
    },
    'Strawberry': {
        'total_days': 7,
        'transfers': [(6,4,1)],
        6: (0, 1),
        4: (1, 4),
        1: (4, 7),
        'color': 'y',
        'size' : (1,1,3,3,5,6,6),
    },
    'Endive': {
        'total_days': 5,
        'transfers': [(0,2,7)],
        0: (0, 1),
        2: (1, 4),
        7: (4,5),
        'color': 'purple',
        'size' : (1,2,4,6,9),
    },
    'Cabbage': {
        'total_days': 6,
        'transfers': [(6, 7, 5)],
        6: (0, 1),
        7: (1, 3),
        5: (3, 6),
        'color': 'orange',
        'size' : (1,2,4,6,8,8),
    },
    'Raddish': {
        'total_days': 5,
        'transfers': [(4, 7)],
        4: (0, 4),
        7: (4, 5),
        'color': 'c',
        'size' : (1,2,5,6,7),
    },
}'''




#big realistic plant data divided by 3
plant_data = {
    'Lettuce': {
        'total_days': 13,
        'transfers': [(0, 1)],
        0: (0, 7),
        1: (7, 13),
        'color': 'b',
        'size' : (1,1,1,3,3,4,4,4,5,5,6,6,7),
    },
    'Fennel': {
        'total_days': 10,
        'transfers': [(1, 0)],
        1: (0, 3),
        0: (3, 10),
        'color': 'g',
        'size' : (1,1,2,3,4,4,6,6,8,10),
    },
    'Marijuana': {
        'total_days': 12,
        'transfers': [(2, 3 ,4 , 5)],
        2: (0, 3),
        3: (3, 5),
        4: (5,9),
        5: (9,12),
        'color': 'r',
        'size' : (1,1,2,2,3,4,6,6,8,7,7,10),
    },
    'Strawberry': {
        'total_days': 7,
        'transfers': [(6,4,1)],
        6: (0, 2),
        4: (2, 4),
        1: (4, 7),
        'color': 'y',
        'size' : (1,2,3,3,4,6,8),
    },
    'Endive': {
        'total_days': 9,
        'transfers': [(0,2,7)],
        0: (0, 3),
        2: (3, 5),
        7: (5,9),
        'color': 'purple',
        'size' : (1,2,2,3,4,5,7,9,10),
    },
    'Cabbage': {
        'total_days': 11,
        'transfers': [(6, 7, 5)],
        6: (0, 5),
        7: (5, 7),
        5: (7, 11),
        'color': 'orange',
        'size' : (1,2,2,3,4,5,6,7,9,10,11),
    },
    'Raddish': {
        'total_days': 19,
        'transfers': [(4, 7)],
        4: (0, 11),
        7: (11, 19),
        'color': 'c',
        'size' : (1,1,2,2,3,4,5,5,6,6,7,7,8,8,9,10,11,12,13),
    },
}
'''
plant_data = {
    'Lettuce': {
        'total_days': 40,
        'transfers': [(0, 1)],
        0: (0, 20),
        1: (20, 40),
        'color': 'b',
        'size' : (1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,7,7,7),
    },
    'Fennel': {
        'total_days': 30,
        'transfers': [(1, 0)],
        1: (0, 10),
        0: (10, 30),
        'color': 'g',
        'size' : (1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4,4,4,6,6,6,6,6,6,8,8,8,8,10),
    },
    'Marijuana': {
        'total_days': 35,
        'transfers': [(2, 3 ,4 , 5)],
        2: (0, 10),
        3: (10, 16),
        4: (16,26),
        5: (26,35),
        'color': 'r',
        'size' : (1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4,4,4,6,6,6,6,6,6,8,7,7,7,10,10,10,10,10,10),
    },
    'Strawberry': {
        'total_days': 20,
        'transfers': [(6,4,1)],
        6: (0, 8),
        4: (8, 12),
        1: (12, 20),
        'color': 'y',
        'size' : (1,1,1,1,2,2,2,3,3,3,3,4,4,6,6,6,7,7,7,8),
    },
    'Endive': {
        'total_days': 28,
        'transfers': [(0,2,7)],
        0: (0, 10),
        2: (10, 15),
        7: (15,28),
        'color': 'purple',
        'size' : (1,1,1,1,2,2,2,3,3,3,3,4,4,6,6,6,6,6,6,6,7,7,7,7,8,9,9,10),
    },
    'Cabbage': {
        'total_days': 33,
        'transfers': [(6, 7, 5)],
        6: (0, 15),
        7: (15, 21),
        5: (21, 33),
        'color': 'orange',
        'size' : (1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,5,5,5,6,6,6,6,6,6,8,8,8,8,10,10,11,11),
    },
    'Raddish': {
        'total_days': 58,
        'transfers': [(4, 7)],
        4: (0, 32),
        7: (32, 58),
        'color': 'c',
        'size' : (1,1,1,1,1,1,1,1,1,2,2,2,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,6,6,6,6,6,6,6,7,7,7,8,8,8,8,9,9,9,9,9,9,10,10,10,10,10,11,13,13),
    },
}'''

"\nplant_data = {\n    'Lettuce': {\n        'total_days': 40,\n        'transfers': [(0, 1)],\n        0: (0, 20),\n        1: (20, 40),\n        'color': 'b',\n        'size' : (1,1,1,1,1,1,1,1,1,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,6,6,6,6,6,7,7,7),\n    },\n    'Fennel': {\n        'total_days': 30,\n        'transfers': [(1, 0)],\n        1: (0, 10),\n        0: (10, 30),\n        'color': 'g',\n        'size' : (1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4,4,4,6,6,6,6,6,6,8,8,8,8,10),\n    },\n    'Marijuana': {\n        'total_days': 35,\n        'transfers': [(2, 3 ,4 , 5)],\n        2: (0, 10),\n        3: (10, 16),\n        4: (16,26),\n        5: (26,35),\n        'color': 'r',\n        'size' : (1,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,4,4,4,6,6,6,6,6,6,8,7,7,7,10,10,10,10,10,10),\n    },\n    'Strawberry': {\n        'total_days': 20,\n        'transfers': [(6,4,1)],\n        6: (0, 8),\n        4: (8, 12),\n        1: (12, 20),\n        'color': 'y',\n        'size' : (1,1,1,1,2,2,2

In [264]:

n_plants = len(plant_data.keys())
size =  horizon / (n_plants-1)
Meta_nodes = {
    'Lettuce': {
        'source' : Node('source',-3, 0),
        'sink' : Node('sink',n_trays * (n_holes_per_tray + 2) + 1, 0)
    },
    'Fennel' :{
        'source' : Node('source',-3, size),
        'sink' : Node('sink', n_trays * (n_holes_per_tray + 2) + 1, size)
    },
    'Marijuana':{
        'source' : Node('source',-3, size*2),
        'sink' : Node('sink',n_trays * (n_holes_per_tray + 2) + 1, size*2)
    },
    'Strawberry' :{
        'source' : Node('source',-3, size*3),
        'sink' : Node('sink', n_trays * (n_holes_per_tray + 2) + 1, size*3)
    },
    'Endive' :{
        'source' : Node('source',-3, size*4),
        'sink' : Node('sink', n_trays * (n_holes_per_tray + 2) + 1, size*4)
    },
    'Cabbage' :{
        'source' : Node('source',-3, size*5),
        'sink' : Node('sink', n_trays * (n_holes_per_tray + 2) + 1, size*5)
    },
    'Raddish' :{
        'source' : Node('source',-3, size*6),
        'sink' : Node('sink', n_trays * (n_holes_per_tray + 2) + 1, size*6)
    },
}

In [265]:
'''
array of tuple (plant data, first day)
'''
plants = []

for plant, val in plant_data.items():
    for day in range(horizon + 1 - val['total_days']) :
        plants.append((plant,day))

In [266]:

def isTrayInPlant(tray, plant) :
    return tray in plant_data[plant]['transfers'][0]

In [267]:
'''
return dictionnary : (plant commodity, bound)
bound = [0,1]
'''
def get_bounds_per_edge(tray, t, plants):
    bounds_dict = dict()
    for plant in plants:
        day = plant[1]
        this_plant = plant[0]
        if isTrayInPlant(tray, this_plant) :
            day_from = day + plant_data[this_plant][tray][0]
            day_to = day + plant_data[this_plant][tray][1] - 1
            bounds_dict[plant] = (day_from <= t <= day_to)*1
        else :
            bounds_dict[plant] = 0
    return bounds_dict

In [268]:
'''
return dictionnary : (plant commodity, size)
'''
def get_sizes_per_edge(tray, t, plants):
    sizes_dict = dict()
    for plant in plants:
        day = plant[1]
        plant_type = plant[0]
        
        sizes_dict[plant] = 0
        
        if isTrayInPlant(tray, plant_type) :
            day_from = day + plant_data[plant_type][tray][0]
            day_to = day + plant_data[plant_type][tray][1] - 1

            if day_from <= t <= day_to :
                sizes_dict[plant] = plant_data[plant_type]['size'][t - day]
        
    return sizes_dict

In [269]:
tray_edges = []
for tray in range(n_trays):
    for hole in range(n_holes_per_tray) :
        for t in range(horizon) :
            node_from = Node('hole',tray, hole, t)
            
            #for hole_to in range(n_holes_per_tray):
            #    node_to = Node('hole',tray, hole_to, t + 1)
            node_to = Node('hole',tray, hole, t + 1)
            edge = Edge(node_from, node_to, get_bounds_per_edge(tray, t, plants), get_sizes_per_edge(tray,t,plants))
            tray_edges.append(edge)

In [270]:
transfer_edges = []
for plant in plants:
    plant_type = plant[0]
    
    for i in range(len(plant_data[plant_type]['transfers'][0]) - 1) :
        
        from_ = plant_data[plant_type]['transfers'][0][i]
        to_ = plant_data[plant_type]['transfers'][0][i + 1]
        
        
        t = plant_data[plant_type][from_][1] + plant[1]
        
        
        for hole in range(n_holes_per_tray):
            node_from = Node('hole',from_, hole, t)

            for hole_to in range(n_holes_per_tray) :
                node_to = Node('hole',to_, hole_to, t+1)
                edge = Edge(node_from, node_to, get_bounds_per_edge(to_, t, plants), get_sizes_per_edge(to_,t,plants))
                transfer_edges.append(edge)

In [271]:
source_edges = []
for plant in plants:
    plant_type = plant[0]
    tray = plant_data[plant_type]['transfers'][0][0]
    bounds = {p: 1*(p == plant) for p in plants}
    
    source = Meta_nodes[plant_type]['source']
    for hole in range(n_holes_per_tray) :
        node_to = Node('hole',tray, hole, plant[1])
        edge = Edge(source,node_to, bounds, 0)
        source_edges.append(edge)

In [272]:
sink_edges = []
for plant in plants:
    plant_type = plant[0]
    size_tray = len(plant_data[plant_type]['transfers'][0])
    tray = plant_data[plant_type]['transfers'][0][size_tray - 1]
    bounds = {p: 1*(p == plant) for p in plants}
    
    sink = Meta_nodes[plant_type]['sink']
    for hole in range(n_holes_per_tray) :
        node_from = Node('hole',tray, hole, plant[1]+plant_data[plant_type]['total_days'])
        edge = Edge(node_from, sink, bounds, 0)
        sink_edges.append(edge)

In [273]:
g = nx.DiGraph()

In [274]:
'''
edge = (node_from, node_to, bounds, size)
size not used here
'''
def add_edge_to_graph(e):
    edge = (e.node_from, e.node_to)
    g.add_edge(*edge, bounds=e.bounds, size=e.size)

In [275]:
for edge in tray_edges :
    add_edge_to_graph(edge)
for edge in transfer_edges :
    add_edge_to_graph(edge)
for edge in source_edges :
    add_edge_to_graph(edge)
for edge in sink_edges :
    add_edge_to_graph(edge)

In [276]:
#def get_y_coordinate(tray,hole) :
#    return tray * (n_holes_per_tray + 2) + hole

In [277]:
def get_node_pos(g):
        pos = dict()
        for node in g.nodes:
            pos[node] = (node.when, node.where)
        return pos

In [278]:
pos = get_node_pos(g)

In [279]:
#fig = plt.figure(figsize=(15, 8))
#nx.draw(g, pos=pos, node_size=60, node_color='red', edgecolors='w', width=.3, linewidths=2, edge_color='grey')

In [280]:
model = grb.Model()

In [281]:
'''
put the bounds on edge for each commodity

put maximum 1 as capacity for all commodities in one edge
'''
a=0
flow_vars = dict()
for u, v, d in g.edges(data=True):
    for key, val in d['bounds'].items():
        #DELETE THE IF TO GO BACK TO NORMAL
        if(val > 0):
            flow_vars[u, v, key] = model.addVar(vtype='INTEGER', name='{}_{}_{}'.format(*(u, v, key)))
            # per-commodity capacity constraints
            model.addConstr(flow_vars[u, v, key] <= val)
        
    # bundle constraints
    bundle = []
    #bundle = [flow_vars[u, v, key] for key in d['bounds'].keys()]
    bundle += [flow_vars[u, v, key] for key in d['bounds'].keys() if (u, v, key) in flow_vars]
    model.addConstr(grb.quicksum(bundle) <= 1)  
 


In [282]:
'''
give all possible pairs of neighbor holes
'''
def pair_of_neighbors(g) :
    pairs = set()
    for n1 in (n1 for n1 in g.nodes if n1.type == 'hole' and n1.where % 2 == 0) :
        for n2 in (n2 for n2 in g.nodes if n1.neighbors(n2)) :
            pairs.add((n1,n2))
    return pairs

In [283]:
'''
Size constraints
'''

for pair in pair_of_neighbors(g) :
    bundle = []
    for e in (e for e in g.in_edges(pair[0], data=True) if e[0].type == 'hole') :
        for c in e[2]['size'].keys():
            if (e[0], pair[0], c) in flow_vars:
                bundle += [flow_vars[e[0], pair[0], c] * e[2]['size'].get(c)]
        #bundle += [flow_vars[e[0], pair[0], c] * e[2]['size'].get(c) for c in e[2]['size'].keys()]
    for e in (e for e in g.in_edges(pair[1], data=True) if e[0].type == 'hole') :
        for c in e[2]['size'].keys():
            if (e[0], pair[1], c) in flow_vars:
                  bundle += [flow_vars[e[0], pair[1], c] * e[2]['size'].get(c)]
        #bundle += [flow_vars[e[0], pair[1], c] * e[2]['size'].get(c) for c in e[2]['size'].keys()]
    model.addConstr(grb.quicksum(bundle) <= max_size)


In [284]:
'''
maximum 1 plant coming from another hole, no transfer + tray allowed

'''
inflow_from_holes = []
for n in (n for n in g.nodes if n.type == 'hole'):
    for e in (e for e in g.in_edges(n) if e[0].type == 'hole') :
        for c in plants :
            if (e[0], e[1], c) in flow_vars :
                inflow_from_holes += [flow_vars[e[0], e[1], c]]
    #inflow_from_holes = grb.quicksum([flow_vars[e[0], e[1], c] 
                                     # for e in g.in_edges(n) if e[0].type == 'hole'
                                     #for c in plants])
    model.addConstr(grb.quicksum(inflow_from_holes) <= 1)
    inflow_from_holes = []
    #model.addConstr(inflow_from_holes <= 1)
            


In [285]:
model.update()

In [286]:
'''
calculate inflow and outflow for each node except sink/sources

inflow == outflow

respect of graph formula
'''



def get_inflow_outflow(g, flow_v, n, c):
    inflow = 0
    outflow = 0
    for e in g.in_edges(n):
        if (e[0], e[1], c) in flow_vars:
            inflow += grb.quicksum([flow_v[e[0], e[1], c]])
    for e in g.out_edges(n):
        if (e[0], e[1], c) in flow_vars:
            outflow += grb.quicksum([flow_v[e[0], e[1],c]])
    return inflow,outflow

for n in (n for n in g.nodes if n.type == 'hole'):
    for c in plants:
        inflow, outflow = get_inflow_outflow(g, flow_vars, n, c)
        model.addConstr(inflow - outflow == 0)


In [287]:
'''
minimize the transfers

'''

'''
factor = 0.01 ;
transfer_bundle = 0
for e in (e for e in g.edges(data=True) if e[0].where != e[1].where) :
    for key in e[2]['bounds'].keys():
        if (e[0], e[1], key) in flow_vars:
            transfer_bundle += grb.quicksum([flow_vars[e[0], e[1], key] * factor])
'''

"\nfactor = 0.01 ;\ntransfer_bundle = 0\nfor e in (e for e in g.edges(data=True) if e[0].where != e[1].where) :\n    for key in e[2]['bounds'].keys():\n        if (e[0], e[1], key) in flow_vars:\n            transfer_bundle += grb.quicksum([flow_vars[e[0], e[1], key] * factor])\n"

In [288]:
'''
add all edges going to sinks

optimize the number of plants produced
'''

a=0
sink_inflow = []
for n in [n for n in g.nodes if n.sink]:
   
    for e in g.in_edges(n):
        for c in plants :
            if (e[0], e[1], c) in flow_vars :
                sink_inflow += [flow_vars[e[0], e[1], c]]

In [289]:
'''
balance the production
'''


alpha = 4
limit = (grb.quicksum(sink_inflow) / n_plants) - alpha
for n in [n for n in g.nodes if n.sink]:
    sink = []
    for e in g.in_edges(n):
        for c in plants:
            if (e[0], e[1], c) in flow_vars :
                sink += [flow_vars[e[0], e[1], c]]
    model.addConstr(grb.quicksum(sink) >= limit)



In [290]:
model.update()

In [291]:
#model.setObjective(grb.quicksum(sink_inflow) - transfer_bundle)
model.setObjective(grb.quicksum(sink_inflow))

In [292]:
model.ModelSense = -1

In [293]:
model.setParam('TimeLimit', 90*60)

Changed value of parameter TimeLimit to 5400.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [294]:
model.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (linux64)
Optimize a model with 2910419 rows, 336840 columns and 2222518 nonzeros
Model fingerprint: 0xefc65d00
Variable types: 0 continuous, 336840 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 2894279 rows and 301335 columns (presolve time = 6s) ...
Presolve removed 2894279 rows and 301335 columns
Presolve time: 7.00s
Presolved: 16140 rows, 35505 columns, 166827 nonzeros
Variable types: 0 continuous, 35505 integer (35505 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   3.109500e+03      8s
    7691    1.3768787e+02   0.000000e+00   7.665496e+03     10s
   13835    1.66

In [186]:
print(horizon)
sol_dict = dict()
for n in [n for n in g.nodes if n.sink]:
    delta_plus = list(g.in_edges(n))
    n_dict = dict()
    for e in delta_plus:
        sum_ = sum([flow_vars[e[0], e[1], c].X for c in plants if (e[0], e[1], c) in flow_vars])
        n_dict[e[0]] = sum_
        sol_dict[n] = n_dict

100


In [ ]:
for e in g.edges:
    # a list of plants that have flow on the edge
    plant_through = []
    for p in plants :
        if((e[0], e[1], p) in flow_vars and flow_vars[e[0], e[1], p].X > 0):
        #if(e[0], e[1], p in flow_vars and flow_vars[e[0], e[1], p].X > 0):
            plant_through += [p]
    #plant_through = [p for p in plants if flow_vars[e[0], e[1], p].X > 0]
    if len(plant_through):
        plant_type = plant_through[0][0]
        g.edges[e[0], e[1]]['color'] = plant_data[plant_type]['color']
    else:
        g.edges[e[0], e[1]]['color'] = 'grey'

In [39]:
# some brute-force plotting

fig = plt.figure(figsize=(20, 12))
limits = plt.axis("off")  # turn off axis

nx.draw_networkx_nodes(g, pos, node_size=60, node_color='red', edgecolors='w', linewidths=2)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'grey']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='grey', width=.3)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'b']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='b', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'g']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='g', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'r']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='r', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'y']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='y', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'purple']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='purple', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'orange']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='orange', width=1)

edgelist = [(e, e_) for e, e_, d in g.edges(data=True) if d['color'] == 'c']
nx.draw_networkx_edges(g, pos, edgelist, edge_color='c', width=1)


plt.show()


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f65deb86710> (for post_execute):


KeyboardInterrupt: 